In [1]:
using CSV, DataFrames, Statistics, Random, Plots, JuMP, Gurobi

In [24]:
using ScikitLearn
@sk_import linear_model: LinearRegression
@sk_import linear_model: Lasso
@sk_import model_selection: train_test_split

┌ Warning: Module model_selection has been ported to Julia - try `import ScikitLearn: CrossValidation` instead
└ @ ScikitLearn.Skcore ~/.julia/packages/ScikitLearn/sqLdT/src/Skcore.jl:259


PyObject <function train_test_split at 0x2ab26dea0>

In [7]:
function opt_split(X, y, n, p, k, lambda, weight = nothing)
    # add column of ones
    X = hcat(ones(Int, size(X, 1)), X)
    p = p + 1

    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)

    @variable(model, theta)
    @variable(model, u[1:n] >= 0)
    @variable(model, beta[1:p])
    @variable(model, w[1:p])

    @objective(model, Min, k * theta + sum(u) + lambda * sum(w))

    for i in 1:p
        @constraint(model, w[i] >= beta[i])
        @constraint(model, w[i] >= -beta[i])
    end 

    for i in 1:n
        if weight == nothing
            @constraint(model, theta + u[i] >= y[i] - sum(X[i, :].*beta))
            @constraint(model, theta + u[i] >= -(y[i] - sum(X[i, :].*beta)))
        else 
            @constraint(model, theta + u[i] >=  weight[i] * (y[i] - sum(X[i, :].*beta)))
            @constraint(model, theta + u[i] >= - weight[i] * (y[i] - sum(X[i, :].*beta)))
        end 
    end
    
    optimize!(model)

    return value(theta), value.(u), value.(beta), value.(w)
end

opt_split (generic function with 2 methods)

In [8]:
function split_train_val(X, y, beta_star, train_fraction = 0.8, weights = nothing)
    # add column of ones
    X = hcat(ones(Int, size(X, 1)), X)

    residuals = y - X * beta_star

    if weights == nothing
        sorted_indices = sortperm(abs.(residuals), rev=true)
    else 
        residuals_weights = [residuals[i] * weights[i] for i in 1:length(weights)]
        sorted_indices = sortperm(abs.(residuals_weights), rev=true)
    end

    num_train_points = round(Int, train_fraction * length(sorted_indices))

    train_indices = sorted_indices[1:num_train_points]

    val_indices = setdiff(1:length(y), train_indices)

    X_train = X[train_indices, 2:end]
    y_train = y[train_indices, :]

    X_val = X[val_indices, 2:end]
    y_val = y[val_indices, :]

    return X_train, y_train, X_val, y_val
end

split_train_val (generic function with 3 methods)

In [9]:
function fit_evaluate_model(X_train, X_val, X_test, y_train, y_val, y_test, lambdas)
    current_lambda = Inf
    current_val_mse = Inf
    current_test_mse = Inf
    
    for lambda in lambdas
        model = Lasso(alpha=lambda)
        fit!(model, X_train, y_train)
        
        y_pred_val = predict(model, X_val)
        val_mse_i = mse(y_val, y_pred_val)
        
        if current_val_mse >= val_mse_i
            current_lambda = lambda
            current_val_mse = val_mse_i
            current_test_mse = mse(y_test, predict(model, X_test))
        end
    end
    
    println("best lambda: ", current_lambda)
    println("val score: ", current_val_mse)
    println("test score: ", current_test_mse)
end

fit_evaluate_model (generic function with 1 method)

In [10]:
function mse(y_true, y_pred)
    mse = mean((y_true .- y_pred).^2)
    return mse
end

mse (generic function with 1 method)

### Abalone dataset

In [25]:
# Load the data
df = CSV.read("abalone_original.csv", DataFrame)
selected_columns = setdiff(names(df), ["rings", "sex"])

X = df[:,selected_columns]
y = CSV.read("abalone_original.csv", DataFrame)[!,"rings"];

(train_X, train_y), (test_X, test_y) =
    IAI.split_data(:regression, X, y, train_proportion=0.9);

In [26]:
#get optimised split 
n = nrow(train_X)
p = ncol(train_X)
train_fraction = 0.7
k = n * train_fraction
lambda = 0.001;

In [27]:
#get optimised split
_, _, betas_star, _ = opt_split(Array(train_X), Array(train_y), n, p, k, lambda)
X_train_opt, y_train_opt, X_val_opt, y_val_opt= split_train_val(Array(train_X), Array(train_y), betas_star);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11


In [28]:
#get non optimised split 
(X_train_noopt, y_train_noopt), (X_val_noopt, y_val_noopt) =
    IAI.split_data(:regression, train_X, train_y, train_proportion=train_fraction);

In [29]:
lambdas = [0.00001, 0.0001, 0.001, 0.01, 0.1]
fit_evaluate_model(X_train_opt, X_val_opt, Matrix(test_X), y_train_opt, y_val_opt, Array(test_y), lambdas)

best lambda: 1.0e-5
val score: 0.2797520418409477
test score: 4.944705216608463


/Users/norahallqvist/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.720e+02, tolerance: 3.570e+00
  model = cd_fast.enet_coordinate_descent(
/Users/norahallqvist/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.556e+00, tolerance: 3.570e+00
  model = cd_fast.enet_coordinate_descent(


In [30]:
fit_evaluate_model(Matrix(X_train_noopt), Matrix(X_val_noopt), Matrix(test_X), Array(y_train_noopt), y_val_noopt, Array(test_y), lambdas)

best lambda: 0.1
val score: 4.946774345220674
test score: 4.966011577831817


/Users/norahallqvist/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.479e+03, tolerance: 2.806e+00
  model = cd_fast.enet_coordinate_descent(
/Users/norahallqvist/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.024e+02, tolerance: 2.806e+00
  model = cd_fast.enet_coordinate_descent(
/Users/norahallqvist/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca